<a href="https://colab.research.google.com/github/rih28/dataAnalytics/blob/master/Tutorial4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/rih28/dataAnalytics/master/tutorial2dnndata.csv', index_col=0)

In [2]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

   Apr  Aug  Dec  Feb  Jan  Jul  ...  Tue  Wed  year  temp  wdsp  NUM_TRIPS
1    0    0    0    0    1    0  ...    0    0  2009  20.0   7.4   0.631126
2    0    0    0    0    1    0  ...    0    0  2009  30.0  10.8   0.724951
3    0    0    0    0    1    0  ...    1    0  2009  30.4   5.7   0.716044
4    0    0    0    0    1    0  ...    0    0  2009  37.9  16.4   0.799994
5    0    0    0    0    1    0  ...    0    0  2009  25.7  13.8   0.872611
6    0    0    0    0    1    0  ...    0    0  2009  21.6   6.9   0.809792

[6 rows x 23 columns]


In [5]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:22]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

     Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sun  Thu  Tue  Wed  year  temp  wdsp
558    0    0    0    0    0    0    0  ...    0    0    1    0  2010  68.1   9.8
584    0    0    0    0    0    0    0  ...    0    1    0    0  2010  55.8   3.1
135    0    0    0    0    0    0    0  ...    0    0    0    0  2009  60.6   6.8
538    0    1    0    0    0    0    0  ...    0    0    0    0  2010  67.2   7.5
268    0    0    0    0    0    0    0  ...    0    0    0    1  2009  42.9   7.3
152    0    0    0    0    0    0    1  ...    0    0    0    0  2009  66.6   6.7

[6 rows x 22 columns]


In [6]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,wdsp,NUM_TRIPS
558,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,2010,68.1,9.8,0.723813
584,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,2010,55.8,3.1,0.822354
135,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,2009,60.6,6.8,0.676365
538,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2010,67.2,7.5,0.790915
268,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,2009,42.9,7.3,0.854527


In [7]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,22]

# print out the first 6 rows of the targets data.
print(targets[:6])

558    0.723813
584    0.822354
135    0.676365
538    0.790915
268    0.854527
152    0.888328
Name: NUM_TRIPS, dtype: float64


In [8]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [9]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_TRIPS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_TRIPS']) - trainsize

# Define the number of input values (predictors)
nppredictors = 22
# Define the number of output values (targets)
noutputs = 1

In [14]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_TRIPS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_TRIPS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f079970d2e8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow

In [18]:
#"Apr","Aug","Dec","Feb","Jan","Jul","Jun","Mar","May","Nov","Oct","Sep","Fri","Mon","Sat","Sun","Thu","Tue","Wed","year","temp","wdsp","NUM_TRIPS"
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [1,0,0],
         'Dec' : [0,1,0],
         'Feb' : [0,0,1],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [1,1,1],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'year' : [2009,2009,2009],
         'temp' : [61.8, 31.2, 40.0],
         'wdsp' : [5.0, 3.0, 8.0]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0799261ac8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model/mo

For the first, you can see below, we are in August, Saturday (we have a monday), 2009. We have a lower temperature and lower windspeed. The difference in day is likely to account for the higher actual number of trips.

205	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	2009	64.9	4.2	0.700748221591537

For second (December):

337	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	35.2	6.6	0.62902717790116

For third (February):

28	0	0	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	38.1	3.6	0.675798566550843

The results overall look reasonable. You can look into this further. 


Considerations for Assignment. I would likely use another method to either standardise or normalise num_trips. And perhaps not bother and scale them in here with SCALE_NUM_TRIPS with a generally large number.

Obviously, you will likely want to have a number of variations but there is no reason you can't use most of the data given. Remember, the DNN is trying to solve a complex relationship, not a linear one. 

Other things to consider when doing this would be to take validation. Of course, in the assignment you will have a much larger range of data i.e. from some date x to y.

This will give you more data. Taking real validation data that hasn't been shown to the model will give real results for you to check against rather than what I have done here (which is just for information).